In [1]:
import pickle
from pprint import pprint
train_docs = pickle.load(open('senti.traindocs.bin','rb'))
test_docs = pickle.load(open('senti.testdocs.bin','rb'))
pprint(train_docs[0])

(['아/Exclamation',
  '더빙/Noun',
  '../Punctuation',
  '진짜/Noun',
  '짜증나다/Adjective',
  '목소리/Noun'],
 '0')


In [2]:
tokens = [t for d in train_docs for t in d[0]]
print(len(tokens))

2159921


In [3]:
print(tokens[0:5])

['아/Exclamation', '더빙/Noun', '../Punctuation', '진짜/Noun', '짜증나다/Adjective']


In [4]:
import nltk
text = nltk.Text(tokens, name='NMSC')
print(text)

<Text: NMSC>


In [5]:
import numpy as np
from sklearn import svm
from scipy import sparse

In [6]:
NUM_FEATURES = 100
NUM_TRAIN = 2000
NUM_TEST = 1000
selected_words = {t[0]:idx for idx, t in enumerate(text.vocab().most_common(NUM_FEATURES))}
train_X = []
def term_vector(doc):
    v = [0]*NUM_FEATURES
    for w in doc:
        if w in selected_words:
            v[selected_words[w]] = 1
    return v
# 
train_X = [term_vector(d) for d, c in train_docs[:NUM_TRAIN]]
train_y = [1 if (c=='1') else 0 for d, c in train_docs[:NUM_TRAIN]]
test_X = [term_vector(d) for d, c in test_docs[:NUM_TEST]]
test_y = [1 if (c=='1') else 0 for d, c in test_docs[:NUM_TEST]]

In [7]:
clf = svm.SVC(gamma='auto', probability=True)
clf.fit(train_X, train_y)

SVC(gamma='auto', probability=True)

In [8]:
from sklearn.metrics import accuracy_score
y_pred = np.argmax(clf.predict_proba(train_X), axis=1)
accuracy_score(train_y, y_pred)

0.719

In [9]:
y_pred = np.argmax(clf.predict_proba(test_X), axis=1)
accuracy_score(test_y, y_pred)

0.693